In [40]:
!pip install datasets unsloth ollama

In [41]:
!pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-03kf0343
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-03kf0343
  Resolved https://github.com/unslothai/unsloth.git to commit 2ff5dc1a8de1614994a275785b7b64fb4db8cb5d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [42]:
import re
import subprocess
import time

import ollama
import pandas as pd
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import (
    FastLanguageModel,
    apply_chat_template,
    is_bfloat16_supported,
    standardize_sharegpt,
    to_sharegpt,
)

In [43]:
RANDOM_STATE = 0

In [44]:
df = pd.read_excel('выгрузка_артикулов.xlsx')

Подготовка инструкции для LLM

Так как наши описания на русском, то и инструкции для LLM сделаем тоже на русском.

In [45]:
def create_instruction_columns(df):
    """
    Создаем столбцы 'instruction', 'input' и 'output' в DataFrame на основе
    столбцов 'text' и 'entity'.

    Аргументы:
    df: pandas DataFrame с колонками 'text' и 'entity'.

    Возвращает:
    pandas DataFrame с добавленными колонками.
    """
    df_new = pd.DataFrame()
    df_new['instruction'] = df['text'].apply(
        lambda description: f'Найди артикул в следующем тексте: [{description}], напиши только артикул.'
    )
    df_new['output'] = df['entity']  # Столбец output содержит сущности

    return df_new

In [46]:
df_train, df_test = train_test_split(df, random_state=RANDOM_STATE, shuffle=True, test_size=0.2)

In [47]:
df_train = create_instruction_columns(df_train)
df_test = create_instruction_columns(df_test)


In [48]:
df_train.head(3)

,instruction,output
6419,Найди артикул в следующем тексте: [Цилиндр гид...,131307430
7146,Найди артикул в следующем тексте: [ВИНТ 001170...,0011706810
185,Найди артикул в следующем тексте: [Ремень бала...,MD310484


In [49]:
df_test.head(3)

,instruction,output
2730,Найди артикул в следующем тексте: [Шланг гидро...,5557Я-3408678
39,Найди артикул в следующем тексте: [~СО2-инкуба...,CO48332001
528,Найди артикул в следующем тексте: [Блок дополн...,LADN11


In [50]:
dataset_train, dataset_test = Dataset.from_pandas(df_train), Dataset.from_pandas(df_test)
dataset_train

Dataset({
    features: ['instruction', 'output', '__index_level_0__'],
    num_rows: 8192
})

Возьмем очень легкую по параметрам модель, из-за ограниченных вычислительных мощностей.

Модель предобучена в том числе и на русском, что нам подойдет.

In [51]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True, # Добавляем квантизацию
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

In [52]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = RANDOM_STATE,
    use_rslora = False,
    loftq_config = None,
)

Подготовка данных

In [53]:
dataset = to_sharegpt(
    dataset_train,
    merged_prompt = "{instruction}",
    output_column_name = "output",
    conversation_extension = 3, # В одном диалоге 3 сообщения
)

Merging columns:   0%|          | 0/8192 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/8192 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/8192 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/8192 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/8192 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/8192 [00:00<?, ? examples/s]

In [54]:
dataset[1]

{'conversations': [{'from': 'human',
   'value': "('Найди артикул в следующем тексте: [ВИНТ 0011706810], напиши только артикул.',)"},
  {'from': 'gpt', 'value': '0011706810'},
  {'from': 'human',
   'value': "('Найди артикул в следующем тексте: [Ремень вентилятора №409619100], напиши только артикул.',)"},
  {'from': 'gpt', 'value': '409619100'},
  {'from': 'human',
   'value': "('Найди артикул в следующем тексте: [Прокладка 840.1012083-20 колпака масляно], напиши только артикул.',)"},
  {'from': 'gpt', 'value': '840.1012083-20'}]}

In [55]:
# Подготовка данных для LLM
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/8192 [00:00<?, ? examples/s]

In [56]:
dataset[0]['conversations']

[{'content': "('Найди артикул в следующем тексте: [Цилиндр гидравлический пресс-фильтра Andritz №131307430], напиши только артикул.',)",
  'role': 'user'},
 {'content': '131307430', 'role': 'assistant'},
 {'content': "('Найди артикул в следующем тексте: [Выключатель разъединитель Compact INS100 4P 100A Schneider Electric (28909)], напиши только артикул.',)",
  'role': 'user'},
 {'content': '28909', 'role': 'assistant'},
 {'content': "('Найди артикул в следующем тексте: [Шайба катка USCO №8T-4994], напиши только артикул.',)",
  'role': 'user'},
 {'content': '8T-4994', 'role': 'assistant'}]

## Customizable Chat Templates

In [57]:
chat_template = """Ниже приведена инструкция, описывающая некоторое задания. Напишите ответы, подходящий для выполнения задания.

### Инструкция:
{INPUT}

### Ответ:
{OUTPUT}"""

dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/8192 [00:00<?, ? examples/s]

In [58]:
dataset[1]

{'conversations': [{'content': "('Найди артикул в следующем тексте: [ВИНТ 0011706810], напиши только артикул.',)",
   'role': 'user'},
  {'content': '0011706810', 'role': 'assistant'},
  {'content': "('Найди артикул в следующем тексте: [Ремень вентилятора №409619100], напиши только артикул.',)",
   'role': 'user'},
  {'content': '409619100', 'role': 'assistant'},
  {'content': "('Найди артикул в следующем тексте: [Прокладка 840.1012083-20 колпака масляно], напиши только артикул.',)",
   'role': 'user'},
  {'content': '840.1012083-20', 'role': 'assistant'}],
 'text': "Ниже приведена инструкция, описывающая некоторое задания. Напишите ответы, подходящий для выполнения задания.\n\n### Инструкция:\n('Найди артикул в следующем тексте: [ВИНТ 0011706810], напиши только артикул.',)\n\n### Ответ:\n0011706810<|endoftext|>\n\n### Инструкция:\n('Найди артикул в следующем тексте: [Ремень вентилятора №409619100], напиши только артикул.',)\n\n### Ответ:\n409619100<|endoftext|>\n\n### Инструкция:\n('Н

Обучение модели

Взяты стандартные параметры, при необходимости можно затюнить их.

In [59]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 50,
        num_train_epochs = 1, # Будем учить одну эпоху
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = RANDOM_STATE,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/8192 [00:00<?, ? examples/s]

The trainer orchestrates the fine-tuning process by combining the model, tokenizer, dataset, and hyperparameters. It ensures efficient training, taking advantage of mixed precision (fp16 or bf16) and memory-efficient optimizations (e.g., AdamW with 8-bit precision). Additionally, it handles sequence preprocessing, gradient accumulation, and learning rate scheduling.

In [60]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,192 | Num Epochs = 1 | Total steps = 1,024
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 3,145,728/4,000,000,000 (0.08% trained)


Step,Training Loss
1,2.286800
2,2.071500
3,2.235200
4,2.107900
5,2.071500
6,2.001400
7,1.825800
8,1.626000
9,1.571400
10,1.579500


## Ollama

OLAMA is a powerful tool that enables you to run large language models (LLMs) directly on your own computer (laptop or desktop).


In [61]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Let's save this model.

In [62]:
model.save_pretrained_gguf("model", tokenizer)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.66 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 43.79it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...


Unsloth: Extending model/tokenizer.model with added_tokens.json.
Originally tokenizer.model is of size (32000).
But we need to extend to sentencepiece vocab size (32011).


INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {3072, 32064}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float16 --> Q8_0, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> Q8_0, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_v.weight,         torch.float16 --> Q8_0, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> Q8_0, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> Q8_0, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> Q8_0, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> Q8_0, 

Подключаем сервер OLAMA

In [63]:
subprocess.Popen(["ollama", "serve"])

time.sleep(5)
print(tokenizer._ollama_modelfile)

FROM {__FILE_LOCATION__}

TEMPLATE """Ниже приведена инструкция, описывающая некоторое задания. Напишите ответы, подходящий для выполнения задания.{{ if .Prompt }}

### Инструкция:
{{ .Prompt }}{{ end }}

### Ответ:
{{ .Response }}<|endoftext|>"""

PARAMETER stop "<|user|>"
PARAMETER stop "<|system|>"
PARAMETER stop "</s>"
PARAMETER stop "<unk>"
PARAMETER stop "<|end|>"
PARAMETER stop "<|assistant|>"
PARAMETER stop "<|endoftext|>"
PARAMETER stop "<|placeholder"
PARAMETER temperature 1.5
PARAMETER min_p 0.1


In [64]:
!ollama create finetuned_model -f ./model/Modelfile

gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠸ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠧ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠼ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠋ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering mode

Посмотрим как работает

In [65]:
response = ollama.chat(model="finetuned_model",
            messages=[{ "role": "user", "content": "Найди артикул в следующем тексте:[Сушилка для белья напольная Mito 20 aluminium (СШИ014)_: НИКА], напиши только сам артикул"
            },
                      ])

print(response.message.content)

СШИ014


Сделаем прогноз сущностей на тесте

In [66]:
predictions = []

for text in df_test['instruction']:
  response = ollama.chat(model="finetuned_model",
            messages=[{ "role": "user", "content": f"Найди артикул в следующем тексте:[{text}], напиши только сам артикул"
            },
                      ])
  prediction = response.message.content
  predictions.append(prediction)

df_test['predictions'] = predictions

In [67]:
df_test

,instruction,output,predictions
2730,Найди артикул в следующем тексте: [Шланг гидро...,5557Я-3408678,5557Я-3408678
39,Найди артикул в следующем тексте: [~СО2-инкуба...,CO48332001,CO48332001
528,Найди артикул в следующем тексте: [Блок дополн...,LADN11,LADN11
2075,Найди артикул в следующем тексте: [Клапан прио...,7505-3435005-20,7505-3435005-20
3896,Найди артикул в следующем тексте: [Набор отвер...,FMHT0-62627,FMHT0-62627
...,...,...,...
5607,Найди артикул в следующем тексте: [Вал привода...,210-1802176,210-1802176
6033,Найди артикул в следующем тексте: [Амфолит 100...,1632094,1632094
7721,Найди артикул в следующем тексте: [Теплообменн...,740.60-1013200,740.60-1013200
6866,Найди артикул в следующем тексте: [Ключ трубны...,15762,15762


In [68]:
df_test.to_csv('predictions_LLM.csv')